In [1]:
import torch
from torch import nn

In [ ]:
'''
CVPRLoss:  使用1-pearson相关性作为loss;
CVPRLoss1: 使用1-spearman rank相关性作为loss;
CVPRLoss_tanh: 使用tanh改写的1-kendall rank相关性作为loss;
CVPRLoss_softsign: 使用softsign改写的1-kendall rank相关性作为loss;
CVPRLoss_erf: 使用erf改写的1-kendall rank相关性作为loss;
CVPRLoss_sigmoid: 使用sigmoid改写的1-kendall rank相关性作为loss;
CVPRLoss_tanh1: 使用tanh改写的1-kendall rank相关性作为loss，矩阵写法;
CVPRLoss_pair： 使用huawei-noah定义的一种rank loss作为loss；
'''
class CVPRLoss(nn.Module):
    # use pearson as loss
    def __call__(self, pred, y):
        pred_mean, y_mean = pred.mean(0), y.mean(0)
        pred_std, y_std = pred.std(0), y.std(0)
        corr = ((pred - pred_mean) * (y - y_mean)).sum(0) / ((((pred - pred_mean)**2).sum(0).sqrt()) * (((y - y_mean)**2).sum(0).sqrt())) 
        return 1-corr

from fast_soft_sort.pytorch_ops import soft_rank
class CVPRLoss1(nn.Module):
    # use spearman as loss refer:https://forum.numer.ai/t/custom-loss-functions-for-xgboost-using-pytorch/960
    def __call__(self, pred, y):
        pred = pred.cpu()
        y = y.cpu()
        return 1-torch.cat(
            [self.spearman(y[:,i].reshape(1,-1), 
                           pred[:,i].reshape(1,-1)).reshape(1,1) for i in range(y.shape[1])]
        ).reshape(1, -1)
        
    def corrcoef(self, target, pred):
        pred_n = pred - pred.mean()
        target_n = target - target.mean()
        pred_n = pred_n / pred_n.norm()
        target_n = target_n / target_n.norm()
        return (pred_n * target_n).sum()


    def spearman(self,
        target,
        pred,
        regularization="l2",
        regularization_strength=0.01,
    ):
        pred = soft_rank(
            pred,
            regularization=regularization,
            regularization_strength=regularization_strength,
        )
        return self.corrcoef(target, pred / pred.shape[-1])
    
class CVPRLoss_tanh(nn.Module):
    # use soft kendall as loss, loop
    def __call__(self, pred, y):
        return 1-torch.cat(
            [self.get_score(y[:,i].reshape(1,-1), 
                           pred[:,i].reshape(1,-1)).reshape(1,-1) for i in range(y.shape[1])]
        ).reshape(1, -1)
    
    def get_score(self, actuals, preds):
        sa = actuals.argsort()[0]
        tmp = preds.index_select(1, sa.int())[0]
        score = torch.cat([((tmp[i:]-tmp[i-1])).tanh() for i in range(1, tmp.shape[0])]).sum()
        score1 = score/sum(list(range(1,len(tmp))))
        return score1
    
class CVPRLoss_tanh1(nn.Module):
    # use soft kendall as loss, mat
    def __call__(self, pred, y):
        return 1-torch.cat(
            [self.get_score(pred[:,i], y[:,i]).reshape(1) for i in range(y.shape[1])]
        ).reshape(1,-1)
    
    def get_score(self, outputs, labels):
        output1 = outputs.unsqueeze(1).repeat(1,outputs.shape[0])
        label1 = labels.unsqueeze(1).repeat(1,labels.shape[0])

        tmp = ((output1-output1.t())*torch.sign(label1-label1.t())).tanh()
        eye_tmp = tmp*torch.eye(tmp.shape[0]).cuda()
        new_tmp = tmp - eye_tmp
        
        loss = torch.sum(new_tmp)/(outputs.shape[0]*(outputs.shape[0]-1))

        return loss
    
    
class CVPRLoss_pair(nn.Module):
    # use rank pair loss refer: https://github.com/huawei-noah/Efficient-Computing/blob/master/Efficient-NAS/ReNAS/renas.py
    def __call__(self, pred, y):
        return torch.cat(
            [self.pair_loss(pred[:,i], y[:,i]).reshape(1) for i in range(y.shape[1])]
        ).reshape(1,-1)
    
    def pair_loss(self, outputs, labels):
        output = outputs.unsqueeze(1)
        output1 = output.repeat(1,outputs.shape[0])
        label = labels.unsqueeze(1)
        label1 = label.repeat(1,labels.shape[0])

        tmp = (output1-output1.t())*torch.sign(label1-label1.t())
        tmp = torch.log(1+torch.exp(-tmp))
        eye_tmp = tmp*torch.eye(len(tmp)).cuda()
        new_tmp = tmp - eye_tmp
        loss = torch.sum(new_tmp)/(outputs.shape[0]*(outputs.shape[0]-1))

        return loss
    
class CVPRLoss_softsign(nn.Module):
    #kendall softsign
    def __call__(self, pred, y):
        return 1-torch.cat(
            [self.get_score(y[:,i].reshape(1,-1), 
                           pred[:,i].reshape(1,-1)).reshape(1,-1) for i in range(y.shape[1])]
        ).reshape(1, -1)

    
    def get_score(self, actuals, preds):
        sa = actuals.argsort()[0]
        tmp = preds.index_select(1, sa.int())[0]
        score = torch.cat([(tmp[i:]-tmp[i-1])/(0.01+torch.abs(tmp[i:]-tmp[i-1])) for i in range(1, tmp.shape[0])]).sum()
        score1 = score/sum(list(range(1,len(tmp))))
        return score1
    
class CVPRLoss_erf(nn.Module):
    # kendall erf
    def __call__(self, pred, y):
        return 1-torch.cat(
            [self.get_score(y[:,i].reshape(1,-1), 
                           pred[:,i].reshape(1,-1)).reshape(1,-1) for i in range(y.shape[1])]
        ).reshape(1, -1)
    
    def get_score(self, actuals, preds):
        sa = actuals.argsort()[0]
        tmp = preds.index_select(1, sa.int())[0]
        score = torch.cat([((tmp[i:]-tmp[i-1])).erf() for i in range(1, tmp.shape[0])]).sum()
        score1 = score/sum(list(range(1,len(tmp))))
        return score1
    
class CVPRLoss_sigmoid(nn.Module):
    # kendall sigmoid
    def __call__(self, pred, y):
        return 1-torch.cat(
            [self.get_score(y[:,i].reshape(1,-1), 
                           pred[:,i].reshape(1,-1)).reshape(1,-1) for i in range(y.shape[1])]
        ).reshape(1, -1)
    
    def get_score(self, actuals, preds):
        sa = actuals.argsort()[0]
        tmp = preds.index_select(1, sa.int())[0]
        score = torch.cat([2*(((tmp[i:]-tmp[i-1])).sigmoid() - 0.5) for i in range(1, tmp.shape[0])]).sum()
        score1 = score/sum(list(range(1,len(tmp))))
        return score1